<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/Image_recommendation_system_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de recomandação por imagens

Projeto desenvolvido para o desafio da DIO sobre recomenadação por imagens. O modelo foi construído a partir do código de [exemplo](https://colab.research.google.com/github/sparsh-ai/rec-tutorials/blob/master/_notebooks/2021-04-27-image-similarity-recommendations.ipynb). O modelo utilizado foi [BiT collection]('https://tfhub.dev/google/bit/m-r50x1/1') da versão 1, que é mais leve que o modelo do exemplo.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Verificação da ativação da GPU

In [ ]:
!nvidia-smi -L

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import keras
import itertools
import matplotlib.pylab as plt
import numpy as np
from shutil import move
from tqdm import tqdm


In [ ]:
!pip install -q -U kaggle
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## Conexão com API kaggle

Identificação do usuário via API kaggle foi carregada para fazer o download do dataset em zip

In [ ]:
with open('/content/kaggle.json', 'r') as file:
    kaggle_creds = json.load(file)

os.environ['KAGGLE_USERNAME'] = kaggle_creds['username']
os.environ['KAGGLE_KEY'] = kaggle_creds['key']

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [ ]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
!unzip fashion-product-images-small.zip

## Estrutura do dataset

O diretório Fashion_data e as imagens foram criados e as imagens foram distribuídas em categorias dentro do Fashion_data

In [ ]:
os.mkdir('/content/Fashion_data')
os.chdir('/content/Fashion_data')

df = pd.read_csv('/content/styles.csv', usecols=['id','masterCategory']).reset_index()
df['id'] = df['id'].astype('str')

all_images = os.listdir('/content/images/')
co = 0
os.mkdir('/content/Fashion_data/categories')
for image in tqdm(all_images):
    category = df[df['id'] == image.split('.')[0]]['masterCategory']
    category = str(list(category)[0])
    if not os.path.exists(os.path.join('/content/Fashion_data/categories', category)):
        os.mkdir(os.path.join('/content/Fashion_data/categories', category))
    path_from = os.path.join('/content/images', image)
    path_to = os.path.join('/content/Fashion_data/categories', category, image)
    move(path_from, path_to)
    co += 1
print('Moved {} images.'.format(co))

## Verificação das categorias e distribuição em gráfico

In [ ]:
df = pd.read_csv('/content/styles.csv', on_bad_lines='skip')
df.head()

In [ ]:
fig = plt.figure(figsize=(15,5))
sns.countplot(x='masterCategory',data=df)

## Versão do Tensorflow e Tensorflow-hub

As versões Tensorflow 2.13.0 e Tensorflow-hub 0.14.0 foram utilizadas para rodar o modelo.

In [ ]:
!pip install tensorflow==2.13.0 tensorflow-hub==0.14.0

In [ ]:
print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)

In [ ]:
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## Upload do modelo

In [ ]:
MODULE_HANDLE = 'https://tfhub.dev/google/bit/m-r50x1/1'
module = hub.load(MODULE_HANDLE)
print("Módulo carregado com sucesso:", module)

In [ ]:
data_dir = '/content/Fashion_data/categories'

## Tratamento das imagens para o modelo

Dimensionamento dos valores de pixel, divisão dos dados em conjuntos de treinamento e validação e, opcionalmente, aplicação de aumentos às imagens de treinamento. Os objetos train_generator e valid_generator são iteradores que fornecerão lotes de imagens para seu modelo durante o treinamento e a avaliação, respectivamente.

In [ ]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

do_data_augmentation = False
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)

## Configuração do modelo de deep learning

Configuração do modelo de Deep Learning para classificação de imagens usando um modelo pré-treinado [BiT collection]('https://tfhub.dev/google/bit/m-r50x1/1'). Personalização do modelo com adição das camadas dropout e densas, aplicada a regularização para evitar overfitting e exibe um resumo da arquitetura com model.summary().

In [ ]:
print("Building model with", MODULE_HANDLE)
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=False),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(N_FEATURES,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

## Definição do optmizer e perda

Configuração dos componentes principais para treinar o modelo. Definição de como o modelo aprenderá (otimizador), o que ele pretende minimizar (função de perda) e como seu desempenho será avaliado (métricas). O cronograma de taxa de aprendizado é adicionado para ajustar dinamicamente a taxa de aprendizado durante o treinamento para melhor convergência.

In [ ]:
lr = 0.01 * BATCH_SIZE / 512
SCHEDULE_LENGTH = 100
SCHEDULE_BOUNDARIES = [200]

# Decay learning rate by a factor of 10 at SCHEDULE_BOUNDARIES.
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=SCHEDULE_BOUNDARIES,
                                                                   values=[lr, lr*0.1])
optimizer = tf.keras.optimizers.AdamW(learning_rate=3e-4, weight_decay=1e-4)

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=['accuracy'])

## Variável checkpoint

A variável checkpoint irá salvar o melhor modelo em best_model.keras com seu mair valor de treinamento.

In [ ]:
checkpoint = ModelCheckpoint(
    filepath='best_model.keras',
    monitor='val_acc',
    save_best_only=True,
    mode='max',
    verbose=1
)


## Treinamento do modelo deep learning

Treinamento do modelo para duas épocas usando os dados de treinamento do train_generator, que será avaliado seu desempenho após cada época com os dados de validação do valid_generator. Os parâmetros steps_per_epoch e validation_steps controlam quantos lotes são usados ​​para treinamento e validação dentro de cada época, respectivamente. O histórico de treinamento (métricas) é salvo na variável history.

In [ ]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
history = model.fit(
    train_generator,
    epochs=2, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    callbacks=[checkpoint],
    validation_steps=validation_steps).history

## Gráfico do período de aprendizado do modelo

In [ ]:
plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(history["loss"])
plt.plot(history["val_loss"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(history["accuracy"])
plt.plot(history["val_accuracy"])

## Salvamento do modelo e extrator no google drive

Modelo treinado e seu extrator de recursos armazenados com segurança no Google Drive para uso futuro.

In [ ]:
if not os.path.exists('/content/gdrive/MyDrive/ImgSim/'):
    os.mkdir('/content/gdrive/MyDrive/ImgSim/')

feature_extractor = tf.keras.Model(inputs=model.inputs, outputs=model.layers[-3].output)
feature_extractor.save('/content/gdrive/MyDrive/ImgSim/bit_feature_extractor', save_format='tf')

saved_model_path = '/content/gdrive/MyDrive/ImgSim/bit_model'
tf.saved_model.save(model, saved_model_path)

## Vetorização das imagens

O vetor de características de cada imagem será salvo como uma matriz em um diretório. Após o processamento, salvaremos esses embeddings para uso posterior.

In [ ]:
img_paths = []
for path in Path('/content/Fashion_data/categories').rglob('*.jpg'):
  img_paths.append(path)
np.random.shuffle(img_paths)

In [ ]:
TRANSFER_LEARNING_FLAG = 1
if TRANSFER_LEARNING_FLAG:
  module = tf.keras.models.load_model('/content/gdrive/MyDrive/ImgSim/bit_feature_extractor')
else:
  module_handle = "https://tfhub.dev/google/bit/m-r50x1/1/ilsvrc2012_classification/1"
  module = hub.load(module_handle)

In [ ]:
imgvec_path = '/content/img_vectors/'
Path(imgvec_path).mkdir(parents=True, exist_ok=True)

## Metadados e Indexação

Será atribuido um id exclusivo a cada imagem e será criado dicionários para localizar informações dessa imagem: 1) Id da imagem para o dicionário de nome da imagem, 2) Id da imagem para o dicionário de vetor de recurso da imagem e 3) (opcional) Id da imagem para o dicionário de id do produto de metadados. Também criaremos um id da imagem para a indexação do vetor de recurso da imagem. Então, salvaremos esses dicionários e objetos de índice para uso posterior.

In [ ]:
import pandas as pd
import glob
import os
import numpy as np
from tqdm import tqdm
tqdm.pandas()
!pip install -q annoy
import json
from annoy import AnnoyIndex
from scipy import spatial
import pickle
from IPython.display import Image as dispImage

## Imagem para teste do dataset

In [ ]:
test_img = '/content/Fashion_data/categories/Accessories/1941.jpg'
dispImage(test_img)

## Verifica se o produto está no dataset

A função match_id recebe um nome de arquivo (fname) como entrada. Ela procura um produto no DataFrame styles cujo ID corresponde ao nome do arquivo. Se encontrar uma correspondência, ela retorna o índice desse produto no DataFrame. Esse índice pode então ser usado para acessar outras informações sobre o produto, como sua categoria, descrição, etc., do DataFrame.

In [ ]:
def match_id(fname):
  return styles.index[styles.id==fname].values[0]

## Configuração das estruturas de dados necessárias para um sistema de busca de similaridade de imagem usando a biblioteca Annoy.

Estruturas de dados: Três dicionários são criados para armazenar o mapeamento entre índices de arquivo de imagem, nomes de arquivo, vetores de recursos e (opcionalmente) IDs de produto. Esses dicionários serão preenchidos posteriormente para recuperar informações de imagem de forma eficiente.

Configuração Annoy: Parâmetros-chave para o índice Annoy são definidos, incluindo a dimensionalidade dos vetores de recursos (dims), o número desejado de vizinhos mais próximos (n_nearest_neighbors) e o número de árvores no índice (árvores). Esses parâmetros influenciam a precisão e o desempenho da busca de similaridade.

Leitura de arquivo: O código identifica e lê os caminhos de todos os arquivos que armazenam vetores de recursos de imagem, que serão usados ​​para construir o índice Annoy.

Inicialização do índice: Um índice Annoy é inicializado com a dimensionalidade especificada e a métrica de distância ('angular'), preparando-o para armazenar e buscar vetores de imagem semelhantes.

In [ ]:
file_index_to_file_name = {}
file_index_to_file_vector = {}
file_index_to_product_id = {}

dims = 256
n_nearest_neighbors = 20
trees = 10000

allfiles = glob.glob('/content/img_vectors/*.npz')

t = AnnoyIndex(dims, metric='angular')

## Preparação dos dados para pesquisa de similaridade de imagem

Carregamento dos vetores de recursos de arquivos, criação dos dicionários para mapear entre IDs de imagem, nomes, vetores e IDs de produtos, e, o mais importante, criação o índice Annoy para pesquisas rápidas de similaridade.

In [ ]:
for findex, fname in tqdm(enumerate(allfiles)):
  file_vector = np.loadtxt(fname)
  file_name = os.path.basename(fname).split('.')[0]
  file_index_to_file_name[findex] = file_name
  file_index_to_file_vector[findex] = file_vector
  try:
    file_index_to_product_id[findex] = match_id(file_name)
  except IndexError:
    pass
  t.add_item(findex, file_vector)

## Contrução do indexes com Annoy

In [ ]:
t.build(trees)
t.save('t.ann')

In [ ]:
file_path = '/content/gdrive/MyDrive/ImgSim/'

## Criação do sistema de mapeamento de imagens por similaridade

In [ ]:
t.save(file_path+'indexer.ann')
pickle.dump(file_index_to_file_name, open(file_path+"file_index_to_file_name.p", "wb"))
pickle.dump(file_index_to_file_vector, open(file_path+"file_index_to_file_vector.p", "wb"))
pickle.dump(file_index_to_product_id, open(file_path+"file_index_to_product_id.p", "wb"))

## Local de teste

Será carregada uma imagem aleatória e será encontrada as K imagens mais semelhantes.

In [ ]:
from PIL import Image
import matplotlib.image as mpimg

## Carregamento da imagem aleatória para teste

a imagem de teste para baixar da web será convertida em um vetor de características usando um codificador de imagem e redimensionará a imagem para exibição, preparando-a para a pesquisa de similaridade de imagens.

In [ ]:
img_addr = 'https://images-na.ssl-images-amazon.com/images/I/81%2Bd6eSA0eL._UL1500_.jpg'

!wget -q -O img.jpg $img_addr
test_img = 'img.jpg'
topK = 4

test_vec = np.squeeze(module(load_img(test_img)))

basewidth = 224
img = Image.open(test_img)
wpercent = (basewidth/float(img.size[0]))
hsize = int((float(img.size[1])*float(wpercent)))
img = img.resize((basewidth,hsize), Image.ANTIALIAS)
img

## Resultado

Criação de um dicionário da pesquisa de imagem, será encontrada as imagens semelhantes usando o índice Annoy e, em seguida, serão exibidas essas imagens semelhantes junto com suas informações de produto em uma figura Matplotlib.

In [ ]:
path_dict = {}
for path in Path('/content/Fashion_data/categories').rglob('*.jpg'):
  path_dict[path.name] = path

nns = t.get_nns_by_vector(test_vec, n=topK)
plt.figure(figsize=(20, 10))
for i in range(topK):
  x = file_index_to_file_name[nns[i]]
  x = path_dict[x+'.jpg']
  y = file_index_to_product_id[nns[i]]
  title = '\n'.join([str(j) for j in list(styles.loc[y].values[-5:])])
  plt.subplot(1, topK, i+1)
  plt.title(title)
  plt.imshow(mpimg.imread(x))
  plt.axis('off')
plt.tight_layout()